In [7]:
# Importação das bibliotecas necessárias
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import numpy as np

In [8]:
# Configuração do webdriver e inicialização do navegador
chrome_service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=chrome_service)

# URL da página
url = 'https://www.imovelweb.com.br/apartamentos-aluguel-distrito-federal.html'

# Carregamento da página
driver.get(url)

# Esperar até que os elementos principais estejam carregados
time.sleep(10)

# Extrair o conteúdo HTML da página
page_content = driver.page_source
soup = BeautifulSoup(page_content, 'html.parser')       

# Fechando o navegador
driver.quit()
soup

<html class="DEFAULT" lang="pt-BR"><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=5.0 shrink-to-fit=no" name="viewport"/>
<title>Apartamentos para alugar no Distrito Federal - Imovelweb</title>
<link href="https://img10.naventcdn.com" rel="preconnect"/>
<link href="https://imgbr.imovelwebcdn.com" rel="preconnect"/>
<script async="" src="https://sb.scorecardresearch.com/c2/14366299/cs.js" type="text/javascript"></script><script async="" src="https://analytics.staticiv.com/xG1igYKHF/iva.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-RY7RHKS235&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" src="//widgets.getsitecontrol.com/97860/script.js"></script><script async="" src="https://tags.creativecdn.com/uB9CJKyi4nRc9QBHv5yw.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-AAAAAAAAAA&amp;l=dataLayer&amp;c

In [9]:
# Encontrar todos os anúncios de imóveis visíveis
anuncios = soup.find_all('div', {'class': 'CardContainer-sc-1tt2vbg-5 fvuHxG'})
print(len(anuncios))

#Visualização da estrutura HTML do primeiro anúncio
anuncios[0]

20


<div class="CardContainer-sc-1tt2vbg-5 fvuHxG"><div class="PostingCardLayout-sc-i1odl-0 egwEUc" data-id="2998758586" data-posting-type="PROPERTY" data-qa="posting PROPERTY" data-to-posting="/propriedades/ap11462-sagocan-reserva-taguatinga-com-02-quartos-2998758586.html"><div class="PostingCardContainer-sc-i1odl-1 kfOVKt"><div class="GalleryContainer-sc-n2cjqs-0 ckhZnU" data-qa="POSTING_CARD_GALLERY"><div class="lazyload-wrapper" style="width: 100%; height: 100%; position: absolute;"><div class="GalleryContainer-sc-1yqjv7m-0 hKvakS"><div class="multimediaGallery flickity-enabled is-draggable" tabindex="0"><div class="flickity-viewport" style="height: 260px; touch-action: pan-y;"><div class="flickity-slider" style="left: 0px; transform: translateX(0%);"><img alt="250d04d2e8eea959cfdc93792cce9e4a1b6088e5 · AP11462 - Sagocan Reserva Taguatinga, com 02 quartos, 69m². Agende a sua visita!" class="is-selected" fetchpriority="high" height="100%" loading="eager" src="https://imgbr.imovelwebcdn.

In [10]:
# Extrair as informações desejadas dos anúncios
imoveis = []
for anuncio in anuncios :
    
    titulo = anuncio.find('a', {'href': 'PostingDescription-sc-i1odl-11 fECErU'})
    
    
    try:
        aluguel_elem = anuncio.find('div', {'class': 'Price-sc-12dh9kl-3 geYYII'})
        aluguel = aluguel_elem.get_text(strip=True) if aluguel_elem else np.nan    
        aluguel = aluguel.replace('R$ ','').replace('.','')
        aluguel = float(aluguel)
    except: 
        aluguel = 0

    try:
        condominio_elem = anuncio.find('div', {'class': 'Expenses-sc-12dh9kl-1 iboaIF'})
        condominio = condominio_elem.get_text(strip=True) if condominio_elem else np.nan    
        condominio = condominio.replace('R$ ','').replace('Condomínio','').replace('Condominio','').replace('.','')
        condominio = re.sub(r'\s.*','',condominio)
        condominio = float(condominio)
    except:
        condominio = 0
    
    bairro_elem = anuncio.find('h2', {'class': 'LocationLocation-sc-ge2uzh-2 fziprF'})
    bairro = bairro_elem.get_text(strip=True) if bairro_elem else np.nan

    area_elem = anuncio.find('h3', {'class':'PostingMainFeaturesBlock-sc-1uhtbxc-0 cHDgeO'})
    area = area_elem.get_text(strip=True) if area_elem else np.nan
        
    imoveis.append({
    'Título': titulo,
    'Bairro': bairro,
    'Aluguel': aluguel,
    'Condomínio': condominio,
    'Área': area
    })

# Exibir os dados extraídos
for imovel in imoveis:
    print(imovel)

{'Título': None, 'Bairro': 'Taguatinga Norte, Taguatinga', 'Aluguel': 2300.0, 'Condomínio': 382.0, 'Área': '69 m² tot.2 quartos1 banheiro1 vaga'}
{'Título': None, 'Bairro': 'Areal, Águas Claras', 'Aluguel': 1700.0, 'Condomínio': 445.0, 'Área': '59 m² tot.2 quartos1 banheiro1 vaga'}
{'Título': None, 'Bairro': 'Aguas Norte, Águas Claras', 'Aluguel': 3600.0, 'Condomínio': 630.0, 'Área': '67 m² tot.3 quartos2 ban.1 vaga'}
{'Título': None, 'Bairro': 'Asa Norte, Brasília', 'Aluguel': 6500.0, 'Condomínio': 690.0, 'Área': '89 m² tot.3 quartos2 ban.'}
{'Título': None, 'Bairro': 'Lago Norte, Brasília', 'Aluguel': 1900.0, 'Condomínio': 438.0, 'Área': '40 m² tot.1 quartos1 banheiro'}
{'Título': None, 'Bairro': 'Aguas Sul, Águas Claras', 'Aluguel': 3500.0, 'Condomínio': 600.0, 'Área': '63 m² tot.2 quartos2 ban.1 vaga'}
{'Título': None, 'Bairro': 'Asa Norte, Brasília', 'Aluguel': 15000.0, 'Condomínio': 3500.0, 'Área': '346 m² tot.4 quartos6 ban.2 vagas'}
{'Título': None, 'Bairro': 'Aguas Norte, Água